In [5]:
# key : CefZja4azgmOQicSgeKjJLi5M0ggLPe66Xx7o3p4
# key : fv3miginwn0fGwBYYLURkEwpK2yaKBfmAIkqOpQU
# key : skzyjdf8GSWoWWUHvNG0ZktTp2kt981bE25jj5V8
#required import requests
import pandas as pd
import requests
from pandas.io.json import json_normalize
import time

In [6]:
#to test to see if you're connecting properly
#params = {'api_key':'fv3miginwn0fGwBYYLURkEwpK2yaKBfmAIkqOpQU'}
params = {'api_key':'skzyjdf8GSWoWWUHvNG0ZktTp2kt981bE25jj5V8', 'search':'receivedate:[2017-01-01 TO 2019-12-31] AND patient.drug.drugindication:Sclerosis AND _exists_:patient.patientonsetage AND _exists_:patient.patientsex AND _exists_:patient.patientweight', 'sort':'receivedate', 'skip':skipstr}
r = requests.get(url, params)
r.json()

NameError: name 'skipstr' is not defined

In [7]:
#returns a count of each condition, in descending order of quantity
#we used the top 3 that weren't unknown
url = "https://api.fda.gov/drug/event.json"
params = {'api_key':'fv3miginwn0fGwBYYLURkEwpK2yaKBfmAIkqOpQU', 'search':'receivedate:[2009-01-01 TO 2019-12-31]', 'count':'patient.drug.drugindication.exact', 'sort':'receivedate'}
data = requests.get(url, params)
data = data.json()
data

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2020-04-03'},
 'results': [{'term': 'PRODUCT USED FOR UNKNOWN INDICATION', 'count': 2556028},
  {'term': 'RHEUMATOID ARTHRITIS', 'count': 534420},
  {'term': 'MULTIPLE SCLEROSIS', 'count': 455416},
  {'term': 'HYPERTENSION', 'count': 322086},
  {'term': 'PSORIASIS', 'count': 231051},
  {'term': 'PAIN', 'count': 228818},
  {'term': 'DIABETES MELLITUS', 'count': 216990},
  {'term': 'PLASMA CELL MYELOMA', 'count': 207484},
  {'term': 'TYPE 2 DIABETES MELLITUS', 'count': 182787},
  {'term': 'DEPRESSION', 'count': 177553},
  {'term': 'PSORIATIC ARTHROPATHY', 'count': 150891},
  {'term': 'CONTRACEPTI

In [27]:
url = "https://api.fda.gov/drug/event.json"

#stands for multiple sclerosis dataframe
msdf = pd.DataFrame()

#this version of the function uses a separate query for each entry. it seems to be the only way to avoid encoding errors
for i in range(2000):
    #this is the number of entries we've already covered, and thus can skip over
    #it has to be a string because that's all params will take
    skip = i
    skipstr = str(skip)
    
    #these are the parameters, the only thing that changes each time is the skip number
    #_exists_ filters out entries that are empty
    #we use "sclerosis" instead of "multiple sclerosis" to avoid other "multiple" conditions.
    params = {'api_key':'fv3miginwn0fGwBYYLURkEwpK2yaKBfmAIkqOpQU', 'search':'receivedate:[2017-01-01 TO 2019-12-31] AND patient.drug.drugindication:Sclerosis AND _exists_:patient.patientsex AND _exists_:patient.patientonsetage AND _exists_:patient.patientweight', 'sort':'receivedate', 'skip':skipstr}
    
    data = requests.get(url, params)
    data = data.json()
    
    #this is the "main" data, which is everything except drug info
    #because drug information is nested oddly, it needs to be extracted into a separate list. json_normalize does not recognize it
    mainitem = json_normalize(data['results'])
    drugdata = json_normalize(mainitem['patient.drug'][0])
    
    
    #then joined
    combined = mainitem.join(drugdata)
    #and then added to the main dataframe
    msdf = msdf.append(combined, sort = False)

    time.sleep(.5)
#showing only the useful columns
#removed seriousnesscongenitalanomali and patient.patientagegroup
msdf_clean = msdf[{'seriousnessdeath', 'companynumb', 'safetyreportid', 'receivedate', 'transmissiondate', 'serious', 'receiptdate', 'patient.patientonsetage', 
   'patient.patientsex', 'patient.patientonsetage', 'patient.patientonsetageunit', 'medicinalproduct', 'patient.patientweight', 'patient.reaction', 'occurcountry', 'seriousnessother', 'seriousnessdisabling', 'seriousnesshospitalization', 'seriousnesslifethreatening'}]


#finally we dump it to a csv
msdf_clean.to_csv('adverseeffects_dataset_multiple_sclerosis.csv')    

In [28]:
combined['medicinalproduct']

0    LATUDA
Name: medicinalproduct, dtype: object

In [10]:
#stands for rhematoid arthritis dataframe
radf = pd.DataFrame()

#this version of the function uses a separate query for each entry. it seems to be the only way to avoid encoding errors
for i in range(2000):
    #this is the number of entries we've already covered, and thus can skip over
    
    #it has to be a string because that's all params will take
    skip = i
    skipstr = str(skip)
    
    #these are the parameters, the only thing that changes each time is the skip number
    #_exists_ filters out entries that are empty
    #note that we use "rheumatoid" instead of "rheumatoid arthritis." this is to avoid other forms of arthritis appearing in the data
    params = {'api_key':'skzyjdf8GSWoWWUHvNG0ZktTp2kt981bE25jj5V8', 'search':'receivedate:[2017-01-01 TO 2019-12-31] AND patient.drug.drugindication:Rheumatoid AND _exists_:patient.patientsex AND _exists_:patient.patientonsetage AND _exists_:patient.patientweight', 'sort':'receivedate', 'skip':skipstr}
    
    data = requests.get(url, params)
    data = data.json()
    
    #this is the "main" data, which is everything except drug info
    #because drug information is nested oddly, it needs to be extracted into a separate list. json_normalize does not recognize it
    mainitem = json_normalize(data['results'])
    drugdata = json_normalize(mainitem['patient.drug'][0])
    
    
    #then joined
    combined = mainitem.join(drugdata)
    

    #and then added to the main dataframe
    radf = radf.append(combined, sort = False)

    time.sleep(.5)

#showing only the useful columns
#removed seriousnesscongenitalanomali and patient.patientagegroup
radf_clean = radf[{'seriousnessdeath', 'companynumb', 'safetyreportid', 'receivedate', 'transmissiondate', 'serious', 'receiptdate', 'patient.patientonsetage', 
   'patient.patientsex', 'patient.patientonsetage', 'patient.patientonsetageunit', 'medicinalproduct', 'patient.patientweight', 'patient.reaction', 'occurcountry', 'seriousnessother', 'seriousnessdisabling', 'seriousnesshospitalization', 'seriousnesslifethreatening'}]


#finally we dump it to a csv
radf_clean.to_csv('adverseeffects_dataset_rheumatoid_arthritis.csv')    

In [13]:
#stands for hypertension dataframe
htdf = pd.DataFrame()

#this version of the function uses a separate query for each entry. it seems to be the only way to avoid encoding errors
for i in range(2000):
    #this is the number of entries we've already covered, and thus can skip over
    #it has to be a string because that's all params will take
    skip = i
    skipstr = str(skip)
    
    #these are the parameters, the only thing that changes each time is the skip number
    #_exists_ filters out entries that are empty
    params = {'api_key':'CefZja4azgmOQicSgeKjJLi5M0ggLPe66Xx7o3p4', 'search':'receivedate:[2017-01-01 TO 2019-12-31] AND patient.drug.drugindication:Hypertension AND _exists_:patient.patientonsetage AND _exists_:patient.patientsex AND _exists_:patient.patientweight', 'sort':'receivedate', 'skip':skipstr}
    
    data = requests.get(url, params)
    data = data.json()
    
    #this is the "main" data, which is everything except drug info
    #because drug information is nested oddly, it needs to be extracted into a separate list. json_normalize does not recognize it
    mainitem = json_normalize(data['results'])
    drugdata = json_normalize(mainitem['patient.drug'][0])
    
    
    #then joined
    combined = mainitem.join(drugdata)
    

    #and then added to the main dataframe
    htdf = htdf.append(combined, sort = False)

    time.sleep(.5)

    
#showing only the useful columns
#removed seriousnesscongenitalanomali and patient.patientagegroup
htdf_clean = htdf[{'seriousnessdeath', 'companynumb', 'safetyreportid', 'receivedate', 'transmissiondate', 'serious', 'receiptdate', 'patient.patientonsetage', 
   'patient.patientsex', 'patient.patientonsetage', 'patient.patientonsetageunit', 'medicinalproduct', 'patient.patientweight', 'patient.reaction', 'occurcountry', 'seriousnessother', 'seriousnessdisabling', 'seriousnesshospitalization', 'seriousnesslifethreatening'}]


#finally we dump it to a csv
htdf_clean.to_csv('adverseeffects_dataset_hypertension.csv')    

In [15]:
htdf_clean = htdf[{'seriousnessdeath', 'companynumb', 'safetyreportid', 'receivedate', 'transmissiondate', 'serious', 'receiptdate', 'patient.patientonsetage', 
   'patient.patientsex', 'patient.patientonsetage', 'patient.patientonsetageunit', 'medicinalproduct', 'patient.patientweight', 'patient.reaction', 'occurcountry', 'seriousnessother', 'seriousnessdisabling', 'seriousnesshospitalization', 'seriousnesslifethreatening'}]


#finally we dump it to a csv
htdf_clean.to_csv('adverseeffects_dataset_hypertension.csv')    